In [1]:
%%HTML 
<style>.container{width:100%;}</style>

Dieses Notebook hängt vom Notebook `Ordered-Sets` ab. Es hängt auch von Dateien im Ordner `Aux` ab, die aus [diesen Vorlesungsunterlagen](https://github.com/karlstroetmann/Logic/tree/master/Python "K. Stroetmann (2020): Algorithms/Python, GitHub") abgeleitet wurden; dem Notebook `N-Queens` und der Datei `davisPutnam.py`, wobei letztere von `N-Queens` importiert wird.

In [2]:
%run Ordered-Sets.ipynb
%run Aux/N-Queens.ipynb

Help on class set in module builtins:

class set(object)
 |  set() -> new empty set object
 |  set(iterable) -> new set object
 |  
 |  Build an unordered collection of unique elements.
 |  
 |  Methods defined here:
 |  
 |  __and__(self, value, /)
 |      Return self&value.
 |  
 |  __contains__(...)
 |      x.__contains__(y) <==> y in x.
 |  
 |  __eq__(self, value, /)
 |      Return self==value.
 |  
 |  __ge__(self, value, /)
 |      Return self>=value.
 |  
 |  __getattribute__(self, name, /)
 |      Return getattr(self, name).
 |  
 |  __gt__(self, value, /)
 |      Return self>value.
 |  
 |  __iand__(self, value, /)
 |      Return self&=value.
 |  
 |  __init__(self, /, *args, **kwargs)
 |      Initialize self.  See help(type(self)) for accurate signature.
 |  
 |  __ior__(self, value, /)
 |      Return self|=value.
 |  
 |  __isub__(self, value, /)
 |      Return self-=value.
 |  
 |  __iter__(self, /)
 |      Implement iter(self).
 |  
 |  __ixor__(self, value, /)
 |      Re

# Performance

## Anwendungsbeispiel: Sieb des Eratosthenes

Als Anwendungsbeispiel benutzen wir das `OrderedSet`, um alle Primzahlen bis zu einem Maximum abzulegen. Die Primzahlen werden dabei mit dem Sieb des Eratosthenes berechnet. Die Implementierung ist sehr einfach gehalten und hat Raum für Optimierungen, aber das ist für den Vergleich mit `set` unproblematisch.

In [3]:
def splay_primes(n):
    primes = OrderedSet(range(2, n + 1))
    for i in range(2, n + 1):
        for j in range(2 * i, n + 1, i):
            primes.discard(j)
    return primes

Wir berechnen probeweise die Primzahlen bis 100.

In [4]:
print(splay_primes(100))

OrderedSet([2, 3, 5, 7, 11, 13, 17, 19, 23, 29, 31, 37, 41, 43, 47, 53, 59, 61, 67, 71, 73, 79, 83, 89, 97])


Wir wiederholen die Implementierung mit dem eingebauten `set`.

In [5]:
def set_primes(n):
    primes = set(range(2, n + 1))
    for i in range(2, n + 1):
        for j in range(2 * i, n + 1, i):
            primes.discard(j)
    return primes

In [6]:
print(set_primes(100))

{2, 3, 5, 7, 11, 13, 17, 19, 23, 29, 31, 37, 41, 43, 47, 53, 59, 61, 67, 71, 73, 79, 83, 89, 97}


Wir betrachten nun vergleichend die Ausführungszeiten beider Varianten.

In [7]:
# Ignore missing whitespace around operator because this is actually a flag
# Ignore syntax error that flake8 sees due to a compatibility issue
n = 10**4
splay_times = %timeit -o splay_primes(n)  # noqa: E225, E999
splay_time = splay_times.best
set_times = %timeit -o set_primes(n)  # noqa: E225
set_time = set_times.best
if splay_time > set_time:
    print(f"Splaying was {round(splay_time / set_time, 2)} times slower")
else:
    print(f"Splaying was {round(set_time / splay_time, 2)} times faster")

1.3 s ± 71.9 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
24.8 ms ± 3.29 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)
Splaying was 51.99 times slower


Dabei ist zu beachten, dass wir zu diesem Zeitpunkt die Python-Implementierung mit einer C-Implementierung vergleichen.

## Anwendungsbeispiel: $n$-Damen-Problem

In einem weiteren Anwendungsbeispiel benutzen wir die geordneten Mengen, um den Algorithmus von Davis und Putnam in der Implementierung von [Stroetmann](https://github.com/karlstroetmann/Logic/tree/master/Python "K. Stroetmann (2020): Algorithms/Python, GitHub") zu implementieren. Damit werden wir das $n$-Damen-Problem lösen, welches als aussagenlogisches Problem aufgefasst wird (Implementierung in der gleichen Quelle). Beim $n$-Damen-Problem ist das Ziel, $n$ Damen auf ein $n \times n$-Schachbrett zu platzieren, sodass keine Dame eine andere Dame gemäß den Regeln des Schachspiels bedroht.

Wir implementieren dafür zunächst alle Methoden des Algorithmus von Davis und Putnam parallel zur existierenden Implementierung, wobei wir das Präfix `splay` benutzen.

In [8]:
def splaySelectVariable(Clauses, Forbidden):
    return dp.arb(OrderedSet([dp.extractVariable(L)
                              for C in Clauses for L in C]) - Forbidden)

def splayReduce(Clauses, l):
    lBar = dp.complement(l)
    return OrderedSet([C - OrderedSet([lBar]) for C in Clauses if lBar in C]) \
        | OrderedSet([C for C in Clauses if lBar not in C and l not in C]) \
        | OrderedSet([OrderedFrozenset([l])])

def splaySaturate(Clauses):
    S = Clauses.copy()
    Units = OrderedSet([C for C in S if len(C) == 1])
    Used = OrderedSet()
    while len(Units) > 0:
        unit = Units.pop()
        Used |= OrderedSet([unit])
        l = dp.arb(unit)  # noqa E471
        S = splayReduce(S, l)
        Units = OrderedSet([C for C in S if len(C) == 1]) - Used
    return S

def splaySolve(Clauses, Variables):
    S = splaySaturate(Clauses)
    empty = OrderedFrozenset()
    Falsum = OrderedSet([empty])
    if empty in S:
        return Falsum
    if all(len(C) == 1 for C in S):
        return S
    p = splaySelectVariable(S, Variables)
    negP = dp.complement(p)
    Result = splaySolve(S | OrderedSet([OrderedFrozenset([p])]),
                        Variables | OrderedSet([p]))
    if Result != Falsum:
        return Result
    return splaySolve(S | OrderedSet([OrderedFrozenset([negP])]),
                      Variables | OrderedSet([p]))

Auch reimplementieren wir die nötigen Methoden für die Spezifikation des $n$-Damen-Problems.

In [9]:
def splayAtMostOne(S):
    return OrderedSet([OrderedFrozenset([('¬', p), ('¬', q)]) for p in S
                       for q in S
                       if p != q])

def splayAtMostOneInRow(row, n):
    return splayAtMostOne(OrderedSet([var(row, col) for col in range(1, n+1)]))

def splayOneInColumn(col, n):
    return OrderedSet([OrderedFrozenset([var(row, col)
                                         for row in range(1, n+1)])])

def splayAtMostOneInFallingDiagonal(k, n):
    S = OrderedSet([var(row, col) for row in range(1, n+1)
                    for col in range(1, n+1)
                    if row - col == k])
    return splayAtMostOne(S)

def splayAtMostOneInRisingDiagonal(k, n):
    S = OrderedSet([var(row, col) for row in range(1, n+1)
                    for col in range(1, n+1)
                    if row + col == k])
    return splayAtMostOne(S)

def splayAllClauses(n):
    All = [splayAtMostOneInRow(row, n) for row in range(1, n+1)] \
        + [splayAtMostOneInRisingDiagonal(k, n) for k in range(3, (2*n-1)+1)] \
        + [splayAtMostOneInFallingDiagonal(k, n)
           for k in range(-(n-2), (n-2)+1)] \
        + [splayOneInColumn(col, n) for col in range(1, n+1)]
    return OrderedSet([clause for S in All for clause in S])

def splayQueens(n):
    Clauses = splayAllClauses(n)
    Solution = splaySolve(Clauses, OrderedSet())
    if Solution != OrderedSet([OrderedFrozenset()]):
        return Solution
    else:
        print(f'The problem is not solvable for {n} queens!')

def splayPrintBoard(I, n):
    if I == OrderedSet([OrderedFrozenset()]):  # noqa E471
        return
    print("-" * (8*n+1))
    for row in range(1, n+1):
        printEmptyLine(n)
        line = "|"
        for col in range(1, n+1):
            if OrderedFrozenset([var(row, col)]) in I:
                line += "   Q   |"
            else:
                line += "       |"
        print(line)
        printEmptyLine(n)
        print("-" * (8*n+1))

So können wir in beiden Varianten das $n$-Damen-Problem lösen.

In [10]:
n = 4
regularSolution = queens(n)
printBoard(regularSolution, n)

---------------------------------
|       |       |       |       |
|       |       |   Q   |       |
|       |       |       |       |
---------------------------------
|       |       |       |       |
|   Q   |       |       |       |
|       |       |       |       |
---------------------------------
|       |       |       |       |
|       |       |       |   Q   |
|       |       |       |       |
---------------------------------
|       |       |       |       |
|       |   Q   |       |       |
|       |       |       |       |
---------------------------------


In [11]:
splayingSolution = splayQueens(n)
splayPrintBoard(splayingSolution, n)

---------------------------------
|       |       |       |       |
|       |   Q   |       |       |
|       |       |       |       |
---------------------------------
|       |       |       |       |
|       |       |       |   Q   |
|       |       |       |       |
---------------------------------
|       |       |       |       |
|   Q   |       |       |       |
|       |       |       |       |
---------------------------------
|       |       |       |       |
|       |       |   Q   |       |
|       |       |       |       |
---------------------------------


In [12]:
n = 5
splay_times = %timeit -o splayQueens(n)  # noqa: E225, E999
splay_time = splay_times.best
set_times = %timeit -o queens(n)  # noqa: E225
set_time = set_times.best
if splay_time > set_time:
    print(f"Splaying was {round(splay_time / set_time, 2)} times slower")
else:
    print(f"Splaying was {round(set_time / splay_time, 2)} times faster")

1.75 s ± 78.4 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
4.86 ms ± 1.05 ms per loop (mean ± std. dev. of 7 runs, 100 loops each)
Splaying was 395.55 times slower
